# Laboratory 1

### Group R. Rotoloni Gabriele n°733421

#### The input and output datasets can be found in the `DatasetsLocation.md` file in the `Datasets` folder.


After indicating the input and output paths, it's possibile to run the notebook altogether and read the results.
This notebook uses the datasets that have been produced by the notebook in the Preproc folder.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
activatedContractsInputPath = "C:\\Users\\rotol\\Downloads\\res\\Transformed_lavori_attivati.csv"
extendedContractsInputPath = "C:\\Users\\rotol\\Downloads\\res\\Transformed_lavori_prorogati.csv"

Create the dataset from the csv files

In [3]:
activatedContracts = pd.read_csv(activatedContractsInputPath)
activatedContracts.head()

,DATA,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,CITTADINANZA
0,2020-09-05,F,50-64,Attività di famiglie e convivenze come datori ...,NESSUN TITOLO DI STUDIO,PROROGABILE,STRANIERO
1,2019-12-07,M,25-49,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,PROROGABILE,ITALIANO
2,2013-05-06,F,15-24,Fabbricazione di altre apparecchiature elettri...,LICENZA MEDIA,PROROGABILE,ITALIANO
3,2010-12-03,F,25-49,Alberghi,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,PROROGABILE,ITALIANO
4,2021-06-04,F,25-49,Rifugi di montagna,LICENZA MEDIA,PROROGABILE,ITALIANO


In [4]:
extendedContracts = pd.read_csv(extendedContractsInputPath)
extendedContracts.head()

,DATA,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,CITTADINANZA
0,2020-10-01,F,25-49,Rifugi di montagna,LICENZA MEDIA,LAVORO INTERMITTENTE,ITALIANO
1,2013-01-03,M,50-64,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,ITALIANO
2,2018-09-10,M,50-64,Rifugi di montagna,LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,ITALIANO
3,2017-10-04,M,25-49,Realizzazione di coperture,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,STRANIERO
4,2010-03-01,M,50-64,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,ITALIANO


### Question: 
#### Which group of people (citizienship, age, sex, level of education) was the most successful at getting its own contract extended?

For this question I will create two dataframes having the education degree as index and multi-index columns for citizienship, sex and age.

The first one (extendedContractsByGroup) will indicate the number of extended contracts for each category. The second one (activatedExtendableContractsByGroup) represents the number of extendable contracts that have been activated by each category.

The result will be a dataframe having as values the division between the first and the second dataframe.

In [5]:
#This is the dataframe for the number of extended contracts for each group of people. 

extendedContractsByGroup = extendedContracts.groupby(['TITOLOSTUDIO','ETA','GENERE','CITTADINANZA']).size()
for i in range(3):
    extendedContractsByGroup = extendedContractsByGroup.unstack()

extendedContractsByGroup = extendedContractsByGroup.fillna(0)
extendedContractsByGroup

CITTADINANZA                                        ITALIANO            \
GENERE                                                     F             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  108634.0  258465.0   
DIPLOMA DI SPECIALIZZAZIONE                           1152.0    3849.0   
DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO                  326.0     977.0   
DIPLOMA UNIVERSITARIO                                 2418.0    9691.0   
LAUREA - Vecchio o nuovo ordinamento                 15172.0  142646.0   
LICENZA ELEMENTARE                                     207.0    2245.0   
LICENZA MEDIA                                        54511.0  229061.0   
MASTER UNIVERSITARIO DI PRIMO LIVELLO                  194.0    1298.0   
NESSUN TITOLO DI STUDIO                              35916.0   98386.0   
TITOLO DI DOTTORE DI RICERCA                           526.0    5245.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   25518.0   63933.0   
TITOLO DI STUDIO POST-LAUREA                            76.0    1408.0   

CITTADINANZA                                                           \
GENERE                                                                  
ETA                                                   50-64 65 o piu'   
TITOLOSTUDIO                                                            
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  35056.0    1642.0   
DIPLOMA DI SPECIALIZZAZIONE                           575.0      21.0   
DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO                  81.0       5.0   
DIPLOMA UNIVERSITARIO                                 449.0      24.0   
LAUREA - Vecchio o nuovo ordinamento                 7844.0     386.0   
LICENZA ELEMENTARE                                   1981.0     114.0   
LICENZA MEDIA                                       66359.0    1481.0   
MASTER UNIVERSITARIO DI PRIMO LIVELLO                  36.0       0.0   
NESSUN TITOLO DI STUDIO                             23364.0    1227.0   
TITOLO DI DOTTORE DI RICERCA                          190.0      35.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...  14232.0     502.0   
TITOLO DI STUDIO POST-LAUREA                           76.0       0.0   

CITTADINANZA                                                            \
GENERE                                                     M             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  126100.0  199939.0   
DIPLOMA DI SPECIALIZZAZIONE                           1045.0    2926.0   
DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO                  341.0     609.0   
DIPLOMA UNIVERSITARIO                                 1124.0    5652.0   
LAUREA - Vecchio o nuovo ordinamento                  5745.0   64593.0   
LICENZA ELEMENTARE                                     524.0    4008.0   
LICENZA MEDIA                                        98945.0  313181.0   
MASTER UNIVERSITARIO DI PRIMO LIVELLO                  210.0     774.0   
NESSUN TITOLO DI STUDIO                              44954.0  113532.0   
TITOLO DI DOTTORE DI RICERCA                           239.0    3071.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   46216.0   69605.0   
TITOLO DI STUDIO POST-LAUREA                            55.0     857.0   

CITTADINANZA                                                            \
GENERE                                                                   
ETA                                                    50-64 65 o piu'   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...   35264.0    5028.0   
DIPLOMA DI SPECIALIZZAZIONE                            563.0      99.0   
DIPLOMA TERZIARIO

In [6]:
#This is the dataframe for the number of activated extendable contracts for each group of people. 

activatedExtendables = activatedContracts[activatedContracts.CONTRATTO == "PROROGABILE"]
activatedExtendableContractsByGroup = activatedExtendables.groupby(['TITOLOSTUDIO','ETA','GENERE','CITTADINANZA']).size()
for i in range(3):
    activatedExtendableContractsByGroup = activatedExtendableContractsByGroup.unstack()

activatedExtendableContractsByGroup = activatedExtendableContractsByGroup.fillna(0)
activatedExtendableContractsByGroup

CITTADINANZA                                        ITALIANO            \
GENERE                                                     F             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  335208.0  737025.0   
DIPLOMA DI SPECIALIZZAZIONE                           6275.0   19657.0   
DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO                  890.0    3674.0   
DIPLOMA UNIVERSITARIO                                 8568.0   29160.0   
LAUREA - Vecchio o nuovo ordinamento                 67947.0  477058.0   
LICENZA ELEMENTARE                                     665.0    5495.0   
LICENZA MEDIA                                       156777.0  504288.0   
MASTER UNIVERSITARIO DI PRIMO LIVELLO                  983.0    4742.0   
NESSUN TITOLO DI STUDIO                             124383.0  367179.0   
TITOLO DI DOTTORE DI RICERCA                           614.0    8725.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   62523.0  150606.0   
TITOLO DI STUDIO POST-LAUREA                           286.0    3847.0   

CITTADINANZA                                                            \
GENERE                                                                   
ETA                                                    50-64 65 o piu'   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  110029.0    5773.0   
DIPLOMA DI SPECIALIZZAZIONE                           3051.0     162.0   
DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO                  699.0      13.0   
DIPLOMA UNIVERSITARIO                                 2072.0      92.0   
LAUREA - Vecchio o nuovo ordinamento                 31848.0    1656.0   
LICENZA ELEMENTARE                                    5013.0     605.0   
LICENZA MEDIA                                       147867.0    6364.0   
MASTER UNIVERSITARIO DI PRIMO LIVELLO                  148.0       5.0   
NESSUN TITOLO DI STUDIO                             101814.0    7924.0   
TITOLO DI DOTTORE DI RICERCA                           346.0      35.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   35967.0    1254.0   
TITOLO DI STUDIO POST-LAUREA                           285.0      16.0   

CITTADINANZA                                                            \
GENERE                                                     M             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  319216.0  607071.0   
DIPLOMA DI SPECIALIZZAZIONE                           5403.0   14398.0   
DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO                  916.0    4419.0   
DIPLOMA UNIVERSITARIO                                 4525.0   19088.0   
LAUREA - Vecchio o nuovo ordinamento                 36992.0  281875.0   
LICENZA ELEMENTARE                                    1522.0    9823.0   
LICENZA MEDIA                                       236588.0  624012.0   
MASTER UNIVERSITARIO DI PRIMO LIVELLO                  623.0    3058.0   
NESSUN TITOLO DI STUDIO                             133805.0  435229.0   
TITOLO DI DOTTORE DI RICERCA                           342.0    8771.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   97105.0  147338.0   
TITOLO DI STUDIO POST-LAUREA                           165.0    2321.0   

CITTADINANZA                                                            \
GENERE                                                                   
ETA                                                    50-64 65 o piu'   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  117101.0   12401.0   
DIPLOMA DI SPECIALIZZAZIONE                           2930.0     307.0   
D

To make the data more compact, I group up all the education degrees obtained after the High school diploma under one single category called "TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO DI RICERCA"

In [7]:
#Sum the values of the rows together
advancedEdRowE = (extendedContractsByGroup.loc["DIPLOMA DI SPECIALIZZAZIONE"] + 
                  extendedContractsByGroup.loc["DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO"] + 
                  extendedContractsByGroup.loc["DIPLOMA UNIVERSITARIO"] + 
                  extendedContractsByGroup.loc["LAUREA - Vecchio o nuovo ordinamento"] + 
                  extendedContractsByGroup.loc["MASTER UNIVERSITARIO DI PRIMO LIVELLO"] + 
                  extendedContractsByGroup.loc["TITOLO DI DOTTORE DI RICERCA"] +
                  extendedContractsByGroup.loc["TITOLO DI STUDIO POST-LAUREA"])
#Add the row
extendedContractsByGroup.loc[len(extendedContractsByGroup)] = advancedEdRowE
extendedContractsByGroup.rename(index={12:"TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO DI RICERCA"}, inplace=True)
#Now i will delete the seven rows that have been combined
extendedContractsByGroup.drop(index=["DIPLOMA DI SPECIALIZZAZIONE","DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO",
                                     "DIPLOMA UNIVERSITARIO","LAUREA - Vecchio o nuovo ordinamento",
                                     "MASTER UNIVERSITARIO DI PRIMO LIVELLO","TITOLO DI DOTTORE DI RICERCA",
                                     "TITOLO DI STUDIO POST-LAUREA"], inplace=True)
extendedContractsByGroup

CITTADINANZA                                        ITALIANO            \
GENERE                                                     F             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  108634.0  258465.0   
LICENZA ELEMENTARE                                     207.0    2245.0   
LICENZA MEDIA                                        54511.0  229061.0   
NESSUN TITOLO DI STUDIO                              35916.0   98386.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   25518.0   63933.0   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...   19864.0  165114.0   

CITTADINANZA                                                           \
GENERE                                                                  
ETA                                                   50-64 65 o piu'   
TITOLOSTUDIO                                                            
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  35056.0    1642.0   
LICENZA ELEMENTARE                                   1981.0     114.0   
LICENZA MEDIA                                       66359.0    1481.0   
NESSUN TITOLO DI STUDIO                             23364.0    1227.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...  14232.0     502.0   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...   9251.0     471.0   

CITTADINANZA                                                            \
GENERE                                                     M             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  126100.0  199939.0   
LICENZA ELEMENTARE                                     524.0    4008.0   
LICENZA MEDIA                                        98945.0  313181.0   
NESSUN TITOLO DI STUDIO                              44954.0  113532.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   46216.0   69605.0   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...    8759.0   78482.0   

CITTADINANZA                                                            \
GENERE                                                                   
ETA                                                    50-64 65 o piu'   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...   35264.0    5028.0   
LICENZA ELEMENTARE                                    4674.0     802.0   
LICENZA MEDIA                                       112992.0    7200.0   
NESSUN TITOLO DI STUDIO                              37190.0    3877.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   15677.0    1759.0   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...    8562.0    2417.0   

CITTADINANZA                                       STRANIERO                   \
GENERE                                                     F                    
ETA                                                    15-24    25-49   50-64   
TITOLOSTUDIO                                                                    
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...    8429.0  26520.0  1734.0   
LICENZA ELEMENTARE                                     272.0   2537.0   300.0   
LICENZA MEDIA                                        13608.0  70102.0  5740.0   
NESSUN TITOLO DI STUDIO                              17519.0  74958.0  7528.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...    4137.0  12143.0   914.0   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...    1093.0  11803.0   755.0   

CITTADINANZA                                                           \
GENERE                                                              M   
ETA                                                65 o piu'    15-24   
TITOLOSTUDIO    

In [8]:
#here I do the same thing for the activated Extendable Contracts
advancedEdRowA = (activatedExtendableContractsByGroup.loc["DIPLOMA DI SPECIALIZZAZIONE"] + 
                  activatedExtendableContractsByGroup.loc["DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO"] + 
                  activatedExtendableContractsByGroup.loc["DIPLOMA UNIVERSITARIO"] + 
                  activatedExtendableContractsByGroup.loc["LAUREA - Vecchio o nuovo ordinamento"] + 
                  activatedExtendableContractsByGroup.loc["MASTER UNIVERSITARIO DI PRIMO LIVELLO"] + 
                  activatedExtendableContractsByGroup.loc["TITOLO DI DOTTORE DI RICERCA"] +
                  activatedExtendableContractsByGroup.loc["TITOLO DI STUDIO POST-LAUREA"])
#Add the row
activatedExtendableContractsByGroup.loc[len(activatedExtendableContractsByGroup)] = advancedEdRowA
activatedExtendableContractsByGroup.rename(index={12:"TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO DI RICERCA"}, 
                                           inplace=True)
#Now i will delete the seven rows that have been combined
activatedExtendableContractsByGroup.drop(index=["DIPLOMA DI SPECIALIZZAZIONE","DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO",
                                     "DIPLOMA UNIVERSITARIO","LAUREA - Vecchio o nuovo ordinamento",
                                     "MASTER UNIVERSITARIO DI PRIMO LIVELLO","TITOLO DI DOTTORE DI RICERCA",
                                     "TITOLO DI STUDIO POST-LAUREA"], inplace=True)
activatedExtendableContractsByGroup

CITTADINANZA                                        ITALIANO            \
GENERE                                                     F             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  335208.0  737025.0   
LICENZA ELEMENTARE                                     665.0    5495.0   
LICENZA MEDIA                                       156777.0  504288.0   
NESSUN TITOLO DI STUDIO                             124383.0  367179.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   62523.0  150606.0   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...   85563.0  546863.0   

CITTADINANZA                                                            \
GENERE                                                                   
ETA                                                    50-64 65 o piu'   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  110029.0    5773.0   
LICENZA ELEMENTARE                                    5013.0     605.0   
LICENZA MEDIA                                       147867.0    6364.0   
NESSUN TITOLO DI STUDIO                             101814.0    7924.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   35967.0    1254.0   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...   38449.0    1979.0   

CITTADINANZA                                                            \
GENERE                                                     M             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  319216.0  607071.0   
LICENZA ELEMENTARE                                    1522.0    9823.0   
LICENZA MEDIA                                       236588.0  624012.0   
NESSUN TITOLO DI STUDIO                             133805.0  435229.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   97105.0  147338.0   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...   48966.0  333930.0   

CITTADINANZA                                                            \
GENERE                                                                   
ETA                                                    50-64 65 o piu'   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  117101.0   12401.0   
LICENZA ELEMENTARE                                   10893.0    3619.0   
LICENZA MEDIA                                       217176.0   21253.0   
NESSUN TITOLO DI STUDIO                             134772.0   14226.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...   36601.0    3641.0   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...   36514.0    5664.0   

CITTADINANZA                                       STRANIERO            \
GENERE                                                     F             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...   22608.0   57750.0   
LICENZA ELEMENTARE                                    1163.0    6948.0   
LICENZA MEDIA                                        34182.0  134743.0   
NESSUN TITOLO DI STUDIO                              67156.0  310963.0   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...    9473.0   34569.0   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...    4454.0   33607.0   

CITTADINANZA                                                           \
GENERE                                                                  
ETA                                                   50-64 65 o piu'   
TITOLOSTUDIO                                                            
DIP

The ratio is now computed

In [9]:
successRatiodf = extendedContractsByGroup/activatedExtendableContractsByGroup
successRatiodf

CITTADINANZA                                        ITALIANO            \
GENERE                                                     F             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  0.324079  0.350687   
LICENZA ELEMENTARE                                  0.311278  0.408553   
LICENZA MEDIA                                       0.347698  0.454227   
NESSUN TITOLO DI STUDIO                             0.288753  0.267951   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...  0.408138  0.424505   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...  0.232156  0.301929   

CITTADINANZA                                                            \
GENERE                                                                   
ETA                                                    50-64 65 o piu'   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  0.318607  0.284428   
LICENZA ELEMENTARE                                  0.395173  0.188430   
LICENZA MEDIA                                       0.448775  0.232715   
NESSUN TITOLO DI STUDIO                             0.229477  0.154846   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...  0.395696  0.400319   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...  0.240604  0.237999   

CITTADINANZA                                                            \
GENERE                                                     M             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  0.395030  0.329350   
LICENZA ELEMENTARE                                  0.344284  0.408022   
LICENZA MEDIA                                       0.418216  0.501883   
NESSUN TITOLO DI STUDIO                             0.335967  0.260856   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...  0.475938  0.472417   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...  0.178879  0.235025   

CITTADINANZA                                                            \
GENERE                                                                   
ETA                                                    50-64 65 o piu'   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  0.301142  0.405451   
LICENZA ELEMENTARE                                  0.429083  0.221608   
LICENZA MEDIA                                       0.520278  0.338776   
NESSUN TITOLO DI STUDIO                             0.275948  0.272529   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...  0.428322  0.483109   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...  0.234485  0.426730   

CITTADINANZA                                       STRANIERO            \
GENERE                                                     F             
ETA                                                    15-24     25-49   
TITOLOSTUDIO                                                             
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE...  0.372833  0.459221   
LICENZA ELEMENTARE                                  0.233878  0.365141   
LICENZA MEDIA                                       0.398104  0.520265   
NESSUN TITOLO DI STUDIO                             0.260870  0.241051   
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOL...  0.436715  0.351268   
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO...  0.245397  0.351207   

CITTADINANZA                                                            \
GENERE                                                                   
ETA                                                    50-64 65 o piu'   
TITOLOSTUDIO                                                             

In [10]:
#the maxium value in this dataset:
successRatiodf.stack(level=[0,1,2]).index[np.argmax(successRatiodf.values)]

('LICENZA MEDIA', 'STRANIERO', 'F', "65 o piu'")

In [11]:
analis = extendedContracts[(extendedContracts.TITOLOSTUDIO == 'LICENZA MEDIA')&
                  (extendedContracts.CITTADINANZA == 'STRANIERO')&
                  (extendedContracts.GENERE == 'F')&
                  (extendedContracts.ETA == "65 o piu'")]

analis.SETTOREECONOMICODETTAGLIO.unique().tolist()

['Attività di famiglie e convivenze come datori di lavoro per personale domestico',
 'Servizi integrati di gestione agli edifici',
 'Alberghi',
 'Pulizia generale (non specializzata) di edifici',
 'Altre attività di servizi per la persona nca',
 'Strutture di assistenza residenziale per anziani e disabili',
 'Altre attività di pulizia nca',
 'Altre strutture di assistenza sociale residenziale',
 'Ristorazione con somministrazione',
 'Produzione di servizi indifferenziati per uso proprio da parte di famiglie e convivenze',
 'Assistenza sociale non residenziale per anziani e disabili',
 'Lavorazione e conservazione di frutta e di ortaggi (esclusi i succhi di frutta e di ortaggi)',
 'Commercio al dettaglio di confezioni per adulti',
 'Movimento merci relativo ad altri trasporti terrestri',
 'Servizi di vigilanza privata',
 'Minimercati ed altri esercizi non specializzati di alimentari vari',
 'Coltivazione di uva',
 'Fabbricazione di motori, generatori e trasformatori elettrici',
 'Altra 

The most "successful" group seems to be older foreigner female worker with a middle school degree. By watching at the types of job that were extended, on one hand, is probable that people in this group do not have interest in looking for a different job after the expiration of the contract, and on the other they are probably good enough for the employer, which is ok with renewing the contract if needed/possible.

Now we can compute other statistic:

- Average ratio for each education degree:

In [12]:
extendedContractsByGroup.sum(axis=1)/activatedExtendableContractsByGroup.sum(axis=1)

TITOLOSTUDIO
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE PERMETTE L'ACCESSO ALL'UNIVERSITA                                          0.351241
LICENZA ELEMENTARE                                                                                                         0.395347
LICENZA MEDIA                                                                                                              0.472748
NESSUN TITOLO DI STUDIO                                                                                                    0.287555
TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOLASTICA ED EXTRA-SCOLASTICA) CHE NON PERMETTE L'ACCESSO ALL'UNIVERSITÀ ()    0.448386
TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO DI RICERCA                                                                 0.271028
dtype: float64

It is interesting to see that the most "successful" categories are the middle school degree and the professional diploma. The middle school is probably related to what I wrote before and the professional diploma is probably due to the fact that one with this degree represents a specific professional figure.

Instead, the least "successful" are the highest education and those without a degree. The first category is probably low because people with high education should see a transformation into a permanent contract instead of a extension. The second category instead represents people that are easly replaceable.

- Average ratio per citizienship:

In [13]:
extendedContractsByGroup.sum(axis=1, level=[0]).sum()/activatedExtendableContractsByGroup.sum(axis=1, level=[0]).sum()

CITTADINANZA
ITALIANO     0.357749
STRANIERO    0.379358
dtype: float64

The citizienship status does not seem to play a big role on whether a contract will be extended or not

- Average ratio per citizienship and gender

In [14]:
extendedContractsByGroup.sum(axis=1, level=[0,1]).sum()/activatedExtendableContractsByGroup.sum(axis=1, level=[0,1]).sum()

CITTADINANZA  GENERE
ITALIANO      F         0.343974
              M         0.371262
STRANIERO     F         0.318146
              M         0.414316
dtype: float64

For both Italians and foreigners, male got their contract extended more frequently

- Average ratio per age groups

In [15]:
extendedContractsByGroup.sum(axis=1, level=[2]).sum()/activatedExtendableContractsByGroup.sum(axis=1, level=[2]).sum()

ETA
15-24        0.362905
25-49        0.364865
50-64        0.358238
65 o piu'    0.297733
dtype: float64

The average ratio is fairly consistent for all age groups except that people older than 65